In [ ]:
import itertools as it
import pickle as pkl
import os
from collections import OrderedDict
%run ../src/sampling_utils.py
%run ../src/shared_utils.py

In [ ]:
disease = "covid19"
prediction_region = "germany"

with open('../data/counties/counties.pkl', "rb") as f:
    counties = pkl.load(f)

data = load_daily_data(disease, prediction_region, counties)
times = uniform_times_by_day(data.index)
locs = uniform_locations_by_county(counties)

def temporal_bfs(x): return bspline_bfs(x, np.array(
    [0, 0, 1, 2, 3, 4, 5]) * 24 * 3600.0, 2)

def spatial_bfs(x): return [gaussian_bf(x, σ)
                            for σ in [6.25, 12.5, 25.0, 50.0]]

num_features = len(temporal_bfs(tt.fmatrix("tmp"))) * len(spatial_bfs(tt.fmatrix("tmp")))

In [ ]:
%%timeit
samp = IAEffectSampler(data, times, locs, temporal_bfs, spatial_bfs, verbose=False)
res = samp(data.index, data.columns)

In [ ]:
%%timeit
ia_bfs = build_ia_bfs(temporal_bfs, spatial_bfs)
res = sample_ia_effects(data, days, counties, ia_bfs, num_features)